In [ ]:
def get_performance_stats(prices, rfr=0, label=None):
    """
    Generate the performance stats for a time-series price data.

    :param prices: pd.Series or np.array
    :param rfr: risk-free returns as pd.Series or np.array (0 by default)
    :param label: strings, column label as string
    :return: stats - pd.dataframe with statistics (performance measures) organized by row with exact labels:
             Tot Ret     - total cumulative return over the whole period
             Avg Ret     - average annualised return
             rfr         - average annualised risk-free return
             Std         - annualised standard deviation
             SR          - annualised Sharpe Ratio
             Skew        - skewness (same frequency as prices)
             Kurt        - kurtosis (same frequency as prices)
             HWM         - High Water Mark price, i.e. the highest price ever achieved
             HWM date    - date of the HWM
             MDD         - Maximum Drawdown as positive proportional loss from prior peak
             Peak date   - Date of the MDD peak
             Trough date - Date of the MDD trough
             Rec date    - Date the asset recovered from Drawdown loss
             MDD dur     - MDD duration in days from peak to recovery date
    """

    month = 12 # Number of month in a year
    # Pre-allocate empty dataframe
    if label is None:
        stats = pd.DataFrame(index=[0])
    else:
        stats = pd.DataFrame(index=[label])

    returns = prices.pct_change()
    last_index = prices.shape[0] - 1  # need - 1 since array referencing starts from 0
    p_last = prices[last_index]
    stats['Total Return'] = (p_last - prices[0]) / prices[0]  # (prices[ prices.shape[0]] / prices[0])-1 does same in 1 step
    stats['Avg Return'] = (1 + stats['Total Return']) ** (12 / len(prices)) - 1
    stats['Rf Rate'] = rfr.mean(axis=0)[0]  # Note the convention for interbank rate is 365 days
    stats['Volatility'] = returns.std() * np.sqrt(month)  # sigma*sqrt(T)
    stats['Sharpe Ratio']  = (stats['Avg Return'] - stats['Rf Rate']) / stats['Volatility'] # Sharpe Ratio
    stats['Skewness'] = returns.skew()  # For normal should be 0
    stats['Kurtosis'] = returns.kurtosis()  # For normal should be 3
    hwm_time = prices.idxmax() # returns argument corresponding to max price (i.e. Timestamp)
    stats['HWM'] = prices.max() # returns max price
    stats['HWM date'] = hwm_time.date()  # converts Timestamp('2018-01-26 00:00:00') to datetime.date(2018, 1, 26)
    dd = prices.cummax() - prices # get all Draw downs: diffs between cumulative max price and current price
    end_mdd  = dd.idxmax()
    start_mdd = prices[:end_mdd].idxmax()
    # Maximum Draw down as positive proportional loss from peak
    stats['MDD'] = 1 - prices[end_mdd] / prices[start_mdd]  # (same as P_start - P_end) / P_start
    stats['Peak Date'] = start_mdd.date()
    stats['Trough Date'] = end_mdd.date()
    bool_p = prices[end_mdd:] > prices[start_mdd] # True/False current price > price of DD peak

    if bool_p.idxmax().date() > bool_p.idxmin().date():
        stats['Recession Date'] = bool_p.idxmax().date()
        stats['MDD Duration'] = (stats['Recession Date'] - stats['Peak Date'])[0].days
        # MDD duration in days from peak to recovery date
    else:
        stats['Recession Date'] = stats['MDD Duration']  = 'Yet to recover'
    return stats